# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [4]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [5]:
df=df=pd.read_csv('processed.csv')
df.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
1,Mount Southington Ski Area,Connecticut,525,425,100,0,0.0,0,0,0,...,51.0,51.0,63.0,55.0,80.0,60.0,60.0,95.0,51.0,0
2,Powder Ridge Park,Connecticut,720,550,170,0,0.0,0,0,0,...,80.0,68.0,80.0,60.0,80.0,45.0,55.0,100.0,40.0,0
3,Ski Sundown,Connecticut,1075,625,450,0,0.0,0,0,0,...,70.0,70.0,84.0,50.0,45.0,62.0,62.0,95.0,66.0,0
4,Chestnut Mountain Resort,Illinois,1040,475,565,0,0.0,0,0,2,...,139.0,139.0,87.0,60.0,50.0,40.0,55.0,112.0,139.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [6]:
# How many states am I dealing with?
df['state'].value_counts()

Michigan         8
Pennsylvania     7
Minnesota        6
New York         6
Wisconsin        4
Connecticut      3
Maine            3
New Hampshire    3
Massachusetts    2
Ohio             2
Indiana          1
Vermont          1
Illinois         1
Alaska           1
Missouri         1
Iowa             1
Name: state, dtype: int64

In [11]:
# turns my state column into a dummy variable
dummy=pd.get_dummies(df['state'])
dummy.head()

,Alaska,Connecticut,Illinois,Indiana,Iowa,Maine,Massachusetts,Michigan,Minnesota,Missouri,New Hampshire,New York,Ohio,Pennsylvania,Vermont,Wisconsin
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#concatenate my two existing dataframes so that I can put my dummy variables to the right of my original df
df = pd.concat([df,dummy], axis=1)
df.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,Massachusetts,Michigan,Minnesota,Missouri,New Hampshire,New York,Ohio,Pennsylvania,Vermont,Wisconsin
0,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mount Southington Ski Area,Connecticut,525,425,100,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Powder Ridge Park,Connecticut,720,550,170,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ski Sundown,Connecticut,1075,625,450,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chestnut Mountain Resort,Illinois,1040,475,565,0,0.0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#drop the state column, add the new indicator columns
df.drop(columns=['state'], axis=1,inplace=True)
df.head()

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Massachusetts,Michigan,Minnesota,Missouri,New Hampshire,New York,Ohio,Pennsylvania,Vermont,Wisconsin
0,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Mount Southington Ski Area,525,425,100,0,0.0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,Powder Ridge Park,720,550,170,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Ski Sundown,1075,625,450,0,0.0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,Chestnut Mountain Resort,1040,475,565,0,0.0,0,0,2,4,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [16]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 

scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [17]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [18]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [19]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = lm.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [20]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

print(explained_variance_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

-0.4167373285318099
10.010898636985129


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [21]:
print(model.intercept_)

58.87138560835734


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [23]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
AW_sort=pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
AW_sort.sort_values(by=['Coefficient'],ascending=False)


,Coefficient
SkiableTerrain_ac,2.706973e+01
Snow Making_ac,2.171878e+01
Vermont,1.099443e+01
base_elev,7.656914e+00
AdultWeekday,6.735822e+00
Runs,6.649632e+00
daysOpenLastYear,5.353252e+00
TerrainParks,4.626641e+00
Minnesota,4.541067e+00
summit_elev,4.351056e+00


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [24]:
df2=df=pd.read_csv('processed.csv')
df2.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
1,Mount Southington Ski Area,Connecticut,525,425,100,0,0.0,0,0,0,...,51.0,51.0,63.0,55.0,80.0,60.0,60.0,95.0,51.0,0
2,Powder Ridge Park,Connecticut,720,550,170,0,0.0,0,0,0,...,80.0,68.0,80.0,60.0,80.0,45.0,55.0,100.0,40.0,0
3,Ski Sundown,Connecticut,1075,625,450,0,0.0,0,0,0,...,70.0,70.0,84.0,50.0,45.0,62.0,62.0,95.0,66.0,0
4,Chestnut Mountain Resort,Illinois,1040,475,565,0,0.0,0,0,2,...,139.0,139.0,87.0,60.0,50.0,40.0,55.0,112.0,139.0,0


In [27]:
df2.drop(['state','summit_elev','base_elev'], axis = 1, inplace=True)
df2.head()

,Name,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,294,0,0.0,0,0,0,1,0,2,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
1,Mount Southington Ski Area,425,0,0.0,0,0,0,2,2,3,...,51.0,51.0,63.0,55.0,80.0,60.0,60.0,95.0,51.0,0
2,Powder Ridge Park,550,0,0.0,0,0,0,1,2,2,...,80.0,68.0,80.0,60.0,80.0,45.0,55.0,100.0,40.0,0
3,Ski Sundown,625,0,0.0,0,0,0,3,0,2,...,70.0,70.0,84.0,50.0,45.0,62.0,62.0,95.0,66.0,0
4,Chestnut Mountain Resort,475,0,0.0,0,0,2,4,0,3,...,139.0,139.0,87.0,60.0,50.0,40.0,55.0,112.0,139.0,0


In [28]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df2.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df2.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 

scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = lm.predict(X_test)

In [29]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

print(explained_variance_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.4786670044106631
5.123796914756681


In [30]:
print(model.intercept_)

56.128094886129695


In [31]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
AW2_sort=pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
AW2_sort.sort_values(by=['Coefficient'],ascending=False)

,Coefficient
SkiableTerrain_ac,7.097615e+00
vertical_drop,6.696938e+00
AdultWeekday,5.175199e+00
Runs,4.579734e+00
NightSkiing_ac,3.212730e+00
total_chairs,1.587098e+00
Snow Making_ac,1.547464e+00
quad,1.332965e+00
clusters,1.290636e+00
fastQuads,1.081561e+00


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

I am choosing my model 2 which correpsonds to this section's model 3. The factors most responsible for driving the model are SkiableTerrain_ac, vertical_drop, AdultWeekday, Runs, NightSkiing_ac, total_chairs, Snow Making_ac, and quads
